In [1]:
print("hello World")

hello World


In [2]:
%matplotlib inline
import scipy as Sci
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from PIL import Image
import os #Obtain the Current Directory

current_directory = os.getcwd()
print(current_directory)



/Users/ibrahim/Desktop/BrainMRI


In [3]:
image = Image.open('Data/Testing/notumor/Te-no_0010.jpg').convert("L")